In [1]:
import math
import pandas as pd
from pandas import Timestamp
import numpy as np
import glob
import os
import datetime

import matplotlib.pyplot as plt
import plotly.graph_objects as go
from IPython.display import display, HTML

from typing import List, Sequence, Tuple
from sklearn.preprocessing import StandardScaler

import stockanalibs

pd.set_option('display.max_columns', None)


In [2]:
ticker = "AAPL"
folder = "Intraday stocks" 

day_to_check = '2025-04-03' # '2025-04-02' or "None" if all dates to be analyzed

In [3]:
pattern = os.path.join(folder, f"{ticker}_*.csv")
files = glob.glob(pattern)

if not files:
    raise FileNotFoundError(f"No files found for ticker {ticker} in {folder}")

# If there are multiple files, you might sort them or choose the first one.
files.sort()  # sorts alphabetically
file_to_read = files[0]

df = pd.read_csv(file_to_read, index_col=0, parse_dates=["datetime"])
df = df[['open', 'high', 'low', 'close', 'volume']]

ask_bid_spread = 0.03 # percent
spread_fraction = ask_bid_spread / 100.0
# Create 'ask' as close price plus the spread fraction and 'bid' as close price minus the spread fraction.
df['ask'] = round(df['close'] * (1 + spread_fraction),4)
df['bid'] = round(df['close'] * (1 - spread_fraction),4)

df

,open,high,low,close,volume,ask,bid
datetime,,,,,,,
2025-01-02 09:00:00,251.9000,251.90,250.6000,251.3600,834,251.4354,251.2846
2025-01-02 09:01:00,251.3700,251.54,251.2600,251.3200,1175,251.3954,251.2446
2025-01-02 09:02:00,251.3700,251.43,251.2100,251.2100,847,251.2854,251.1346
2025-01-02 09:03:00,251.3000,251.30,250.9800,250.9800,1692,251.0553,250.9047
2025-01-02 09:04:00,250.9600,251.15,250.9400,251.0300,633,251.1053,250.9547
...,...,...,...,...,...,...,...
2025-06-03 23:55:00,203.0000,203.00,202.8635,202.9900,755,203.0509,202.9291
2025-06-03 23:56:00,202.9725,202.99,202.9201,202.9201,531,202.9810,202.8592
2025-06-03 23:57:00,202.9700,202.97,202.9300,202.9699,222,203.0308,202.9090


In [4]:
display(HTML("""
<style>
.output_scroll {
    overflow-y: visible !important;
    max-height: none !important;
}
</style>
"""))


In [5]:
def plot_trades(df, trades, buy_threshold, performance_stats, trade_color="green"):
    """
    Plots the overall close-price series plus trade intervals and two continuous signals,
    with the signals shown on a secondary y-axis.

    • The base trace (grey) plots the close-price series on the primary y-axis.
    • Trade traces (green by default) indicate the intervals for each trade from the original trade list.
    • A dotted blue line shows the raw normalized "signal" on a secondary y-axis.
    • A dashed red line shows the smooth normalized signal on the secondary y-axis.
    • A horizontal dotted line is drawn at the buy_threshold.
    • Additionally, areas between each buy and sell event determined by the new 
      "trade_action" field (buy=+1, sell=-1) are highlighted (in orange).
    • An update menu is added with two buttons:
         - "Hide Trades": Hides only the trade-specific traces.
         - "Show Trades": Makes all traces visible.

    Parameters:
      df : pd.DataFrame
          DataFrame with a datetime index and at least the columns "close", "signal_norm",
          "signal_smooth_norm", and "trade_action".
      trades : list
          A list of tuples, each in the form:
            ((buy_date, sell_date), (buy_price, sell_price), profit_pc).
      buy_threshold : float
          The threshold used for candidate buy detection (shown as a horizontal dotted line on the 
          secondary y-axis).
      performance_stats : dict, optional
          Dictionary containing performance metrics. If provided and if it contains keys
          "Trade Gains ($)" and "Trade Gains (%)" (each a list), they will be added to the
          trade annotations. 
      trade_color : str, optional
          The color to use for the original trade traces.
    """
    fig = go.Figure()
    
    # Trace 0: Base close-price trace.
    fig.add_trace(go.Scatter(
        x=df.index,
        y=df['close'],
        mode='lines',
        line=dict(color='grey', width=1),
        name='Close Price',
        hoverinfo='x+y',
        hovertemplate="Date: %{x}<br>Close: %{y:.2f}<extra></extra>",
    ))
    
    # Trade traces: one per original trade.
    for i, trade in enumerate(trades):
        # Unpack the trade tuple: ((buy_date, sell_date), (buy_price, sell_price), profit_pc)
        (buy_date, sell_date), (_, _), trade_return = trade
        trade_df = df.loc[buy_date:sell_date]
        fig.add_trace(go.Scatter(
            x=trade_df.index,
            y=trade_df['close'],
            mode='lines+markers',
            line=dict(color=trade_color, width=3),
            marker=dict(size=4, color=trade_color),
            name=f"Trade {i+1}",
            hoveron='points',
            hovertemplate=f"Trade {i+1}: Return: {trade_return:.2f}%<extra></extra>",
            visible=True
        ))
        
    # --------------------------------------------------------------------
    # New Trade Action Highlights: using the 'trade_action' field.
    # Extract rows where trade_action is not zero.
    trade_events = df[df["trade_action"] != 0]["trade_action"]
    pairs = []
    prev_buy = None
    for timestamp, action in trade_events.items():
        if action == 1:   # Buy signal
            prev_buy = timestamp
        elif action == -1 and prev_buy is not None:
            pairs.append((prev_buy, timestamp))
            prev_buy = None
    # For each buy-sell pair, add a vertical shaded region with annotation.
    for j, (buy_ts, sell_ts) in enumerate(pairs):
        if (performance_stats is not None and 
            "Trade Gains ($)" in performance_stats and 
            "Trade Gains (%)" in performance_stats and 
            len(performance_stats["Trade Gains ($)"]) > j and 
            len(performance_stats["Trade Gains (%)"]) > j):
            ann_text = (f"TA Trade {j+1}<br>$: {performance_stats['Trade Gains ($)'][j]}<br>"
                        f"%: {performance_stats['Trade Gains (%)'][j]}")
        else:
            ann_text = f"TA Trade {j+1}"
            
        fig.add_vrect(
            x0=buy_ts, x1=sell_ts,
            fillcolor="orange", opacity=0.25,
            line_width=0,
            annotation_text=ann_text,
            annotation_position="top left",
            annotation_font_color="orange"
        )
    # --------------------------------------------------------------------
    
    # Raw Signal trace: Plot the normalized "signal" on a secondary y-axis.
    fig.add_trace(go.Scatter(
        x=df.index,
        y=df['signal_norm'],
        mode='lines',
        line=dict(color='blue', width=2, dash='dot'),
        name='Signal (Normalized)',
        hovertemplate="Date: %{x}<br>Signal: %{y:.2f}<extra></extra>",
        visible=True,
        yaxis="y2"
    ))
    
    # Smooth Signal trace: Plot the smooth normalized signal on a secondary y-axis.
    fig.add_trace(go.Scatter(
        x=df.index,
        y=df['signal_smooth_norm'],
        mode='lines',
        line=dict(color='red', width=2, dash='dash'),
        name='Smooth Signal (Normalized)',
        hovertemplate="Date: %{x}<br>Smooth Signal: %{y:.2f}<extra></extra>",
        visible=True,
        yaxis="y2"
    ))
    
    # Add a horizontal dotted line for the buy_threshold (on secondary y-axis).
    fig.add_hline(y=buy_threshold, line=dict(color="purple", dash="dot"),
                  annotation_text="Buy Threshold", annotation_position="top left", yref="y2")
    
    # Total traces: 1 Base + n_trades (original trades) + 2 (for the signal traces).
    n_trades = len(trades)
    total_traces = 1 + n_trades + 2
    vis_show = [True] * total_traces  
    vis_hide = [True] + ["legendonly"] * n_trades + [True, True]
    
    fig.update_layout(
        updatemenus=[
            {
                "type": "buttons",
                "direction": "left",
                "buttons": [
                    {
                        "label": "Hide Trades",
                        "method": "update",
                        "args": [{"visible": vis_hide}],
                    },
                    {
                        "label": "Show Trades",
                        "method": "update",
                        "args": [{"visible": vis_show}],
                    },
                ],
                "pad": {"r": 10, "t": 10},
                "showactive": True,
                "x": 0.9,
                "xanchor": "left",
                "y": 1.1,
                "yanchor": "top",
            }
        ],
        hovermode="x unified",
        template="plotly_white",
        title="Close Price, Trade Intervals, and Signals",
        xaxis_title="Datetime",
        yaxis_title="Close Price",
        height=700,
        yaxis2=dict(
            title="Signal (Normalized)",
            overlaying="y",
            side="right",
            showgrid=False,
        )
    )
    
    fig.show()


In [6]:
def saveDF_checkDAY(df, ticker, min_prof_thr, max_down_prop, smooth_win_sig, pre_entry_decay, buy_threshold, trailing_stop_thresh, 
                    keep_cols = ["open", "high", "low", "close", "volume", "bid", "ask", 
                                 "trade_action", "StrategyEarning", "EarningDiff", "signal_smooth_norm"], 
                    day_to_check=None):
    
    df = stockanalibs.smooth_prepost_trading_data(df=df, 
                                                 regular_start=stockanalibs.regular_start, 
                                                 regular_end=stockanalibs.regular_end)

    results_by_day = stockanalibs.identify_trades_daily(df=df,
                                                       min_prof_thr=min_prof_thr, 
                                                       max_down_prop=max_down_prop,
                                                       regular_start_shifted=stockanalibs.regular_start_shifted,
                                                       regular_end=stockanalibs.regular_end,
                                                       day_to_check=day_to_check)

    results_by_day = stockanalibs.add_trade_signal_to_results(results_by_day=results_by_day, 
                                                            min_prof_thr=min_prof_thr, 
                                                            regular_start=stockanalibs.regular_start,
                                                            smooth_win_sig=smooth_win_sig, 
                                                            pre_entry_decay=pre_entry_decay,
                                                            buy_threshold=buy_threshold, 
                                                            trailing_stop_thresh=trailing_stop_thresh)

    results_all = stockanalibs.simulate_trading(results_by_day=results_by_day, 
                                               regular_start=stockanalibs.regular_start, 
                                               regular_end=stockanalibs.regular_end,
                                               ticker=ticker)

    if day_to_check is not None: # visualize results for the day selected

        results_to_check = results_all.get(pd.to_datetime(day_to_check))
        
        df_to_check = results_to_check[0]
            
        trade_to_check = results_to_check[1]
        
        performance_to_check = results_to_check[2]
        
        plot_trades(df=df_to_check, 
                    trades=trade_to_check,
                    buy_threshold=buy_threshold,
                    performance_stats=performance_to_check)
    
        print(performance_to_check)

        return df_to_check

    else: # returns the concatenation of the list of all daily dataframes
        
        dfs = [result[0] for result in results_all.values()] 
        
        merged = (pd.concat(dfs)[keep_cols].sort_index())

        merged.to_csv(f"dfs training/merged_{ticker}.csv", index=True)

        return merged

In [7]:
min_prof_thr, max_down_prop, smooth_win_sig, pre_entry_decay, buy_threshold, trailing_stop_thresh = stockanalibs.signal_parameters(ticker)

In [8]:
df_fin = saveDF_checkDAY(
                        df=df,
                        ticker=ticker,
                        min_prof_thr=min_prof_thr, 
                        max_down_prop=max_down_prop, 
                        smooth_win_sig=smooth_win_sig, 
                        pre_entry_decay=pre_entry_decay, 
                        buy_threshold=buy_threshold, 
                        trailing_stop_thresh=trailing_stop_thresh,
                        day_to_check=day_to_check
                        )

df_fin

{'Final Net Value ($)': 7.866, 'Buy & Hold Gain ($)': -4.028, 'Strategy Profit Difference ($)': 11.894, 'Final Net Return (%)': 3.8, 'Buy & Hold Return (%)': -1.946, 'Strategy Improvement (%)': 5.745, 'Trade Gains ($)': [-0.909, 1.673, 0.463, 1.987, 0.418, 0.533, 2.267, -0.093, -0.002, 1.529], 'Trade Gains (%)': [-0.439, 0.825, 0.228, 0.975, 0.205, 0.261, 1.111, -0.045, -0.001, 0.759]}


,open,high,low,close,volume,ask,bid,open_orig,high_orig,low_orig,close_orig,volume_orig,ask_orig,bid_orig,signal,signal_smooth,signal_norm,signal_smooth_norm,trade_action,Position,Cash,NetValue,Action,TradedAmount,StrategyEarning,BuyHoldEarning,EarningDiff
2025-04-03 13:30:00,207.1494,207.2647,207.0100,207.1241,13676.0,207.1862,207.0619,206.7500,207.000,206.5300,206.9000,8684,206.9621,206.8379,0.0,0.00000,0.0,0.000000,0,0,0.000,0.000,No trade,0,0.000,0.000,0.000
2025-04-03 13:31:00,207.1230,207.2347,206.9859,207.1023,12580.0,207.1645,207.0402,206.9600,207.040,206.8500,206.9700,3823,207.0321,206.9079,0.0,0.00000,0.0,0.000000,0,0,0.000,0.000,No trade,0,0.000,0.000,0.000
2025-04-03 13:32:00,207.1017,207.2059,206.9418,207.0635,13159.0,207.1256,207.0014,206.9683,206.990,206.5000,206.5900,24072,206.6520,206.5280,0.0,0.00000,0.0,0.000000,0,0,0.000,0.000,No trade,0,0.000,0.000,0.000
2025-04-03 13:33:00,207.0587,207.1659,206.8941,207.0135,13732.0,207.0756,206.9514,206.5300,206.650,206.4000,206.4000,21568,206.4619,206.3381,0.0,0.00000,0.0,0.000000,0,0,0.000,0.000,No trade,0,0.000,0.000,0.000
2025-04-03 13:34:00,207.0128,207.1212,206.8483,206.9723,13307.0,207.0344,206.9102,206.4700,206.500,206.2000,206.4599,19380,206.5218,206.3980,0.0,0.00000,0.0,0.000000,0,0,0.000,0.000,No trade,0,0.000,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-03 20:56:00,203.6650,203.8600,203.2600,203.3800,766015.0,203.4410,203.3190,203.6650,203.860,203.2600,203.3800,766015,203.4410,203.3190,0.0,0.40625,0.0,0.167267,0,1,-195.134,8.185,Hold,0,8.185,-3.708,11.893
2025-04-03 20:57:00,203.3900,203.5350,203.3239,203.3300,701062.0,203.3910,203.2690,203.3900,203.535,203.3239,203.3300,701062,203.3910,203.2690,0.0,0.14625,0.0,0.060216,0,1,-195.134,8.135,Hold,0,8.135,-3.758,11.893
2025-04-03 20:58:00,203.3300,203.5900,203.3300,203.5100,777503.0,203.5711,203.4489,203.3300,203.590,203.3300,203.5100,777503,203.5711,203.4489,0.0,0.00000,0.0,0.000000,0,1,-195.134,8.315,Hold,0,8.315,-3.578,11.893
2025-04-03 20:59:00,203.5050,203.5100,202.7000,203.0600,1980376.0,203.1209,202.9991,203.5050,203.510,202.7000,203.0600,1980376,203.1209,202.9991,0.0,0.00000,0.0,0.000000,-1,0,7.866,7.866,Sell,-1,7.866,-4.028,11.894
